# **Resize images into 640px**
 
 `C:\Users\CorneAI\YOLOv11_Mobius\eyelid_image\1-295_periocular\images` フォルダ内の画像を640px四方にリサイズし、`C:\Users\CorneAI\YOLOv11_Mobius\eyelid_image\1-295_periocular_640px\images` に保存します。
 
 もし出力先のフォルダ (`C:\Users\CorneAI\YOLOv11_Mobius\eyelid_image\1-295_periocular_640px\images`) が既に存在する場合は、`shutil` を使用して一度削除し、新たに作成してから画像を保存します。

In [1]:
import os
import shutil
from PIL import Image
from tqdm import tqdm

# 入力フォルダと出力フォルダのパス
input_folder = r"C:\Users\CorneAI\YOLOv11_Mobius\eyelid_image\1-295_periocular\images"
output_folder = r"C:\Users\CorneAI\YOLOv11_Mobius\eyelid_image\1-295_periocular_640px\images"
target_size = (640, 640)

# 出力フォルダが存在する場合は削除し、再作成する
if os.path.exists(output_folder):
    shutil.rmtree(output_folder)
os.makedirs(output_folder)

# 入力フォルダ内のファイルリストを取得
file_list = os.listdir(input_folder)

# 入力フォルダ内の画像を処理
for filename in tqdm(file_list, desc="画像の処理中"):
    input_path = os.path.join(input_folder, filename)
    output_path = os.path.join(output_folder, filename)

    # 画像ファイルかどうかを確認 (拡張子で簡易的に判断)
    if filename.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.gif')):
        try:
            # 画像を開く
            img = Image.open(input_path)
            
            # 画像をリサイズ
            resized_img = img.resize(target_size)
            
            # リサイズした画像を保存
            resized_img.save(output_path)
            # tqdm.write(f"Resized and saved: {filename}") # 個別の成功メッセージはプログレスバーで十分なためコメントアウトも検討
        except Exception as e:
            tqdm.write(f"Error processing {filename}: {e}")

print(f"すべての画像の処理が完了しました。出力先: {output_folder}")


画像の処理中: 100%|██████████| 4469/4469 [01:06<00:00, 66.82it/s] 

すべての画像の処理が完了しました。出力先: C:\Users\CorneAI\YOLOv11_Mobius\eyelid_image\1-295_periocular_640px\images


# **Re-calculate and modify CVAT xml**

In [1]:
import xml.etree.ElementTree as ET
import os
import copy # 要素を安全にコピーするために使用
import zipfile # ZIPファイル作成のために追加

# --- 設定 ---
orig_xml_path = r"C:\Users\CorneAI\YOLOv11_Mobius\eyelid_image\1-295_periocular\annotations_1-139.xml" # 元のCVAT XML
dest_xml_path = r"C:\Users\CorneAI\YOLOv11_Mobius\eyelid_image\1-295_periocular\annotations_1-139_640px.xml" # 新たに作成するXML
# orig_image_dir はXML内のwidth/heightを使用するため、座標修正ロジックでは直接使用しませんが、
# 元のコンテキストを維持するために残しておきます。
orig_image_dir = r"C:\Users\CorneAI\YOLOv11_Mobius\eyelid_image\1-295_periocular\images"
dst_img_size = 640 # リサイズ後の目標画像サイズ (正方形を想定)

ZIP_compression = True # XMLファイルをZIP圧縮するかどうか
# --- 設定ここまで ---

def parse_points_from_cvat(points_str):
    """CVATのpoints属性文字列を (x,y) タプルのリストに変換"""
    points = []
    if not points_str:
        return points
    pairs = points_str.split(';')
    for pair in pairs:
        try:
            x_str, y_str = pair.split(',')
            points.append((float(x_str), float(y_str)))
        except ValueError:
            # ポイントペアの解析に失敗した場合の警告（例：空の文字列や不正な形式）
            print(f"警告: ポイントペア '{pair}' の解析に失敗しました (元文字列: '{points_str}')。")
            continue
    return points

def format_points_for_cvat(points_list):
    """(x,y) タプルのリストをCVATのpoints属性文字列に変換"""
    if points_list is None or len(points_list) == 0:
        return ""
    # 各座標を "x.xx,y.yy" 形式の文字列にし、セミコロンで結合
    return ";".join([f"{x:.2f},{y:.2f}" for x, y in points_list])

def main():
    print("--- ワークフロー開始 (全アノテーション座標のリサイズ) ---")
    print(f"入力XML: {orig_xml_path}")
    print(f"出力XML: {dest_xml_path}")
    print(f"目標画像サイズ: {dst_img_size}x{dst_img_size}")
    if ZIP_compression:
        print("ZIP圧縮: 有効")
    print("-" * 30)

    # --- 1. 元のXMLをパース ---
    try:
        print("元のXMLをパース中...")
        if not os.path.exists(orig_xml_path):
            raise FileNotFoundError(f"元のXMLファイルが見つかりません: {orig_xml_path}")
        original_tree = ET.parse(orig_xml_path)
        original_root = original_tree.getroot()
        print("元のXMLをパースしました。")
    except FileNotFoundError as e:
        print(e)
        return
    except ET.ParseError as e:
        print(f"XMLのパースエラー: {e}")
        return

    # --- 2. 新しいXML構造を作成 ---
    print("新しいXML構造を作成中...")
    new_root = ET.Element(original_root.tag) # <annotations>

    # <version> と <meta> をディープコピーで安全にコピー
    version_element = original_root.find('version')
    if version_element is not None:
        new_root.append(copy.deepcopy(version_element))
    meta_element = original_root.find('meta')
    if meta_element is not None:
        new_root.append(copy.deepcopy(meta_element))
    else:
        print("警告: <meta> タグが元のXMLに見つかりません。")

    # --- 3. 画像とアノテーションを処理 ---
    images_processed_count = 0      # 新しいXMLに追加された<image>要素の数
    annotations_resized_count = 0   # 座標がリサイズされたアノテーションの総数
    images_skipped_count = 0        # 必須情報の欠如やエラーでスキップされた画像の数
    total_images_in_xml = len(original_root.findall('image'))

    print(f"元のXML内の {total_images_in_xml} 個の画像を処理中...")
    for image_element in original_root.findall('image'):
        image_id_str = image_element.get('id')
        image_name = image_element.get('name', 'N/A') # name属性がない場合も考慮

        if image_id_str is None:
            print(f"警告: 'id'属性のない<image>要素をスキップします (名前: {image_name})。")
            images_skipped_count +=1
            continue

        try:
            # --- 元の画像サイズを取得 ---
            original_width_str = image_element.get('width')
            original_height_str = image_element.get('height')

            if not all([original_width_str, original_height_str]):
                print(f"  警告: 画像ID {image_id_str} (名前: {image_name}) の width または height 属性がありません。この画像をスキップします。")
                images_skipped_count += 1
                continue

            original_width = int(original_width_str)
            original_height = int(original_height_str)

            if original_width <= 0 or original_height <= 0: # 0または負のサイズは無効
                print(f"  警告: 画像ID {image_id_str} (名前: {image_name}) の width または height が無効です ({original_width}x{original_height})。この画像をスキップします。")
                images_skipped_count += 1
                continue

            # --- 新しい<image>要素を作成し、属性を更新 ---
            # 元の属性をコピーし、widthとheightを新しいサイズに更新
            new_image_attribs = image_element.attrib.copy()
            new_image_attribs['width'] = str(dst_img_size)
            new_image_attribs['height'] = str(dst_img_size)
            new_image_element = ET.Element('image', attrib=new_image_attribs)

            num_resized_annotations_in_image = 0
            for ann_element in image_element: # <polygon>, <box> などのアノテーション子要素を処理
                new_ann_element = copy.deepcopy(ann_element) # タグと他の属性を安全にコピー

                current_ann_resized = False
                if ann_element.tag == 'polygon':
                    points_str = ann_element.get('points')
                    if points_str:
                        parsed_points = parse_points_from_cvat(points_str)
                        if parsed_points: # パース成功時のみ処理
                            scaled_points = []
                            for x, y in parsed_points:
                                new_x = (x / original_width) * dst_img_size
                                new_y = (y / original_height) * dst_img_size
                                scaled_points.append((new_x, new_y))
                            new_ann_element.set('points', format_points_for_cvat(scaled_points))
                            current_ann_resized = True
                elif ann_element.tag == 'box':
                    try:
                        xtl = float(ann_element.get('xtl'))
                        ytl = float(ann_element.get('ytl'))
                        xbr = float(ann_element.get('xbr'))
                        ybr = float(ann_element.get('ybr'))

                        new_xtl = (xtl / original_width) * dst_img_size
                        new_ytl = (ytl / original_height) * dst_img_size
                        new_xbr = (xbr / original_width) * dst_img_size
                        new_ybr = (ybr / original_height) * dst_img_size

                        new_ann_element.set('xtl', f"{new_xtl:.2f}")
                        new_ann_element.set('ytl', f"{new_ytl:.2f}")
                        new_ann_element.set('xbr', f"{new_xbr:.2f}")
                        new_ann_element.set('ybr', f"{new_ybr:.2f}")
                        current_ann_resized = True
                    except (TypeError, ValueError) as e: # get()がNoneを返すか、float変換できない場合
                         print(f"  警告: 画像ID {image_id_str} のboxアノテーションの座標取得/変換エラー: {e}。このアノテーションは変更されずにコピーされます。")
                # 他のタイプのアノテーション（例: <tag>, <polyline>, <points>など）はそのままコピーされる

                if current_ann_resized:
                    annotations_resized_count += 1
                    num_resized_annotations_in_image +=1
                new_image_element.append(new_ann_element)
            
            if num_resized_annotations_in_image > 0:
                 print(f"  画像ID {image_id_str} (名前: {image_name}): {num_resized_annotations_in_image} 個のアノテーション座標をリサイズしました。")
            elif len(list(image_element)) > 0 : # アノテーション要素はあったが、対象(polygon/box)でなかったか、座標情報がなかった場合
                 print(f"  画像ID {image_id_str} (名前: {image_name}): リサイズ対象のアノテーション (有効な座標を持つpolygon/box) が見つかりませんでした。元のアノテーションをコピーしました。")
            # else: 画像にアノテーションが元々ない場合は何も表示しない

            new_root.append(new_image_element)
            images_processed_count += 1

        except ValueError as e: # int(width_str) などでのエラー
            print(f"警告: 画像要素ID {image_id_str} (名前: {image_name}) の処理中にエラー (ValueError): {e}。この画像をスキップします。")
            images_skipped_count += 1
        except Exception as e:
            print(f"予期せぬエラーが画像ID {image_id_str} (名前: {image_name}) の処理中に発生しました: {e}。この画像をスキップします。")
            images_skipped_count += 1

    print("-" * 30)
    print(f"処理完了。サマリー:")
    print(f"元のXMLの総画像数: {total_images_in_xml}")
    print(f"新しいXMLに含まれる画像数: {images_processed_count}")
    print(f"リサイズされたアノテーションの総数 (polygon/box): {annotations_resized_count}")
    print(f"エラー等でスキップされた画像数: {images_skipped_count}")

    # --- 4. 新しいXMLを保存し、オプションでZIP圧縮 ---
    if images_processed_count > 0:
        print(f"新しいアノテーションを保存中: {dest_xml_path}")
        new_tree = ET.ElementTree(new_root)

        try:
            ET.indent(new_tree, space="  ", level=0) # Python 3.9+ で利用可能
        except AttributeError:
            print("警告: ET.indent が利用できません (Python 3.9+ が必要です)。XMLは整形されずに出力されます。")

        dest_dir = os.path.dirname(dest_xml_path)
        if dest_dir and not os.path.exists(dest_dir): # 出力先ディレクトリが存在しない場合は作成
            os.makedirs(dest_dir)
            print(f"出力先ディレクトリを作成しました: {dest_dir}")

        xml_saved = False
        try:
            new_tree.write(dest_xml_path, encoding='utf-8', xml_declaration=True)
            print("新しいXMLファイルが正常に保存されました。")
            xml_saved = True
        except Exception as e:
            print(f"XMLファイルの書き込みエラー: {e}")

        # XMLが正常に保存され、ZIP圧縮が有効な場合にZIPファイルを作成
        if xml_saved and ZIP_compression:
            zip_file_path = os.path.splitext(dest_xml_path)[0] + ".zip" # XMLファイル名から拡張子を除き .zip を付加
            print(f"ZIPアーカイブを作成中: {zip_file_path}")
            try:
                with zipfile.ZipFile(zip_file_path, 'w', zipfile.ZIP_DEFLATED) as zipf:
                    # ZIPアーカイブにXMLファイルを追加 (アーカイブ内ではファイル名のみ)
                    zipf.write(dest_xml_path, arcname=os.path.basename(dest_xml_path))
                print("ZIPアーカイブが正常に作成されました。")
            except Exception as e:
                print(f"ZIPファイルの作成エラー: {e}")
    else:
        print("新しいXMLに含めるために処理された画像要素がありません。出力ファイルは生成されませんでした。")

    print("--- ワークフロー終了 ---")

if __name__ == "__main__":
    main()

--- ワークフロー開始 (全アノテーション座標のリサイズ) ---
入力XML: C:\Users\CorneAI\YOLOv11_Mobius\eyelid_image\1-295_periocular\annotations_1-139.xml
出力XML: C:\Users\CorneAI\YOLOv11_Mobius\eyelid_image\1-295_periocular\annotations_1-139_640px.xml
目標画像サイズ: 640x640
ZIP圧縮: 有効
------------------------------
元のXMLをパース中...
元のXMLをパースしました。
新しいXML構造を作成中...
元のXML内の 4464 個の画像を処理中...
  画像ID 0 (名前: 1-20141126-38-091804_eb568e2ac952f8be45ec0ac9ae800120b7c988b60ac499ca87306986d218f554_L.jpg): 2 個のアノテーション座標をリサイズしました。
  画像ID 1 (名前: 1-20141126-38-091804_eb568e2ac952f8be45ec0ac9ae800120b7c988b60ac499ca87306986d218f554_R.jpg): 2 個のアノテーション座標をリサイズしました。
  画像ID 2 (名前: 1-20150121-38-142903_6e60b2355e174936406b708cf171e424300de779d4b8ae8e3aebb1a9de9905e6_L.jpg): 2 個のアノテーション座標をリサイズしました。
  画像ID 3 (名前: 1-20150121-38-142903_6e60b2355e174936406b708cf171e424300de779d4b8ae8e3aebb1a9de9905e6_R.jpg): 2 個のアノテーション座標をリサイズしました。
  画像ID 4 (名前: 1-20150121-38-142903_f27c8cf98eef934c557ffb70fed94b3fb9d400f29f472f2687b7fcec96e92c77_L.jpg): 2 個のアノテーション座標